In [1]:
from datasets import load_dataset
import torch
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '4'

In [3]:
data = load_dataset(
    path="../../biomedical/bigbio/hub/hub_repos/symptemist/symptemist.py", 
    name="symptemist_entities_bigbio_kb"
)
data

Found cached dataset symptemist (/home/Ignacio.Rodriguez/.cache/huggingface/datasets/symptemist/symptemist_entities_bigbio_kb/1.0.0/6eca3229d6e85bf4857c66298013fa7ce6ad7a5e9376e32f68dadbbf5cd93ecb)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 744
    })
})

In [4]:
data["train"][150]

{'id': '150',
 'document_id': 'es-S1130-01082009000900015-1',
 'passages': [{'id': '150_es-S1130-01082009000900015-1_passage',
   'type': 'clinical_case',
   'text': ['Varón de 54 años con episodios de pancreatitis aguda de repetición, que como antecedentes personales presenta hiperlipemia en tratamiento y fistulotomía previa.\nSeis años antes presentó un episodio de pancreatitis aguda moderada-severa que requirió ingreso. Hace 10 meses presentó un nuevo episodio en el que precisó ERCP, a causa de la cual repitió el cuadro de pancreatitis aguda (intervalo de 5 días).\nSe realizaron como pruebas complementarias una colangiorresonancia magnética donde aparece una pancreatitis crónica sobre páncreas divisum y una atrofia del parénquima hepático, y una ERCP donde también aparecen signos de pancreatitis crónica y páncreas divisum.\n\nSe decide intervención quirúrgica, realizándose pancreatectomía caudal y derivación pancreático-yeyunal. Se encuentra tumoración en cara anterior gástrica, que

In [5]:
label2id = {
    'O': 0,
    -100: -100,
    'B-SINTOMA': 1,
    'I-SINTOMA': 2,
}

id2label = {
    0: 'O',
    1: 'B-SINTOMA',
    2: 'I-SINTOMA',
}

from transformers import AutoTokenizer
model = 'PlanTL-GOB-ES/roberta-base-biomedical-clinical-es'
tokenizer = AutoTokenizer.from_pretrained(model)

In [6]:
import sys
sys.path.append("..")
from src.preprocess import *

In [7]:
symp = tokenize_split(data["train"], label2id, tokenizer)

Tokenizing dataset split: 100%|███████████████████████████████████████████████████████████████████| 744/744 [00:01<00:00, 404.57it/s]


In [9]:
type(symp)

datasets.arrow_dataset.Dataset

In [8]:
i = 100
j = 0
k = j + 50
pd.DataFrame(symp[i]["tokens"][j:k],symp[i]["labels"][j:k])

,0
-100,<s>
0,ĠNiÃ±o
0,Ġde
0,Ġ13
0,ĠaÃ±os
0,Ġreferido
0,Ġal
0,ĠServicio
0,Ġde
0,ĠCirugÃŃa


In [9]:
from transformers import Trainer, TrainingArguments, AutoModelForTokenClassification, DataCollatorForTokenClassification
from src.evaluation import compute_fair

args = TrainingArguments(
    output_dir = "../checkpoints",
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    fp16 = True,
    save_strategy = "epoch",
    overwrite_output_dir = True,
    evaluation_strategy = "epoch",
    save_total_limit = 2,
    num_train_epochs = 40,
    load_best_model_at_end = True,
    metric_for_best_model = "eval_overall_f1",
    lr_scheduler_type = "linear",
    learning_rate = 0.00005,
    warmup_ratio = 0.0,
    label_smoothing_factor = 0.0,
    weight_decay = 0.0,
)

trainer = Trainer(
    args = args,
    model = AutoModelForTokenClassification.from_pretrained(model, id2label = id2label, num_labels = len(id2label)),
    train_dataset=symp.select(range(595)),
    eval_dataset = symp.select(range(595,744)),
    data_collator = DataCollatorForTokenClassification(tokenizer),
    tokenizer = tokenizer,
    compute_metrics = compute_fair(list(id2label.values())),
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import wandb

In [11]:
with wandb.init(project=f'symptemist_ner'):
    trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: illorca. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Overall F1,Overall Fair F1,Faireval,Seqeval
1,No log,0.340474,0.000000,0.000000,"{'SINTOMA': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'trad_prec': 0.0, 'trad_rec': 0.0, 'trad_f1': 0.0, 'TP': 0, 'FP': 0.0, 'FN': 1490.0, 'LE': 0.0, 'BE': 0.0, 'LBE': 0.0}, 'overall_precision': 0.0, 'overall_recall': 0.0, 'overall_f1': 0.0, 'overall_trad_prec': 0.0, 'overall_trad_rec': 0.0, 'overall_trad_f1': 0.0, 'TP': 0, 'FP': 0.0, 'FN': 1490.0, 'LE': 0.0, 'BE': 0.0, 'LBE': 0.0}","{'SINTOMA': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1490}, 'overall_precision': 0.0, 'overall_recall': 0.0, 'overall_f1': 0.0, 'overall_accuracy': 0.8748063868438666}"
2,No log,0.202659,0.375350,0.474294,"{'SINTOMA': {'precision': 0.6183574879227053, 'recall': 0.38467317806160783, 'f1': 0.47429365446966193, 'trad_prec': 0.5059288537549407, 'trad_rec': 0.3436241610738255, 'trad_f1': 0.4092725819344524, 'TP': 512, 'FP': 129.0, 'FN': 632.0, 'LE': 0.0, 'BE': 374.0, 'LBE': 0.0}, 'overall_precision': 0.6183574879227053, 'overall_recall': 0.38467317806160783, 'overall_f1': 0.47429365446966193, 'overall_trad_prec': 0.5059288537549407, 'overall_trad_rec': 0.3436241610738255, 'overall_trad_f1': 0.4092725819344524, 'TP': 512, 'FP': 129.0, 'FN': 632.0, 'LE': 0.0, 'BE': 374.0, 'LBE': 0.0}","{'SINTOMA': {'precision': 0.3923865300146413, 'recall': 0.3597315436241611, 'f1': 0.37535014005602235, 'number': 1490}, 'overall_precision': 0.3923865300146413, 'overall_recall': 0.3597315436241611, 'overall_f1': 0.37535014005602235, 'overall_accuracy': 0.9240845205086529}"
3,No log,0.161459,0.534562,0.647177,"{'SINTOMA': {'precision': 0.7103296703296703, 'recall': 0.5943361529974255, 'f1': 0.6471766119343212, 'trad_prec': 0.6196319018404908, 'trad_rec': 0.5422818791946309, 'trad_f1': 0.5783822476735863, 'TP': 808, 'FP': 158.0, 'FN': 380.0, 'LE': 0.0, 'BE': 343.0, 'LBE': 0.0}, 'overall_precision': 0.7103296703296703, 'overall_recall': 0.5943361529974255, 'overall_f1': 0.6471766119343212, 'overall_trad_prec': 0.6196319018404908, 'overall_trad_rec': 0.5422818791946309, 'overall_trad_f1': 0.5783822476735863, 'TP': 808, 'FP': 158.0, 'FN': 380.0, 'LE': 0.0, 'BE': 343.0, 'LBE': 0.0}","{'SINTOMA': {'precision': 0.524547803617571, 'recall': 0.5449664429530201, 'f1': 0.5345622119815668, 'number': 1490}, 'overall_precision': 0.524547803617571, 'overall_recall': 0.5449664429530201, 'overall_f1': 0.5345622119815668, 'overall_accuracy': 0.9412372119705517}"
4,No log,0.156534,0.583933,0.700217,"{'SINTOMA': {'precision': 0.682909604519774, 'recall': 0.7184249628528975, 'f1': 0.7002172338884867, 'trad_prec': 0.6066499372647428, 'trad_rec': 0.648993288590604, 'trad_f1': 0.6271076523994811, 'TP': 967, 'FP': 264.0, 'FN': 194.0, 'LE': 0.0, 'BE': 370.0, 'LBE': 0.0}, 'overall_precision': 0.682909604519774, 'overall_recall': 0.7184249628528975, 'overall_f1': 0.7002172338884867, 'overall_trad_prec': 0.6066499372647428, 'overall_trad_rec': 0.648993288590604, 'overall_trad_f1': 0.6271076523994811, 'TP': 967, 'FP': 264.0, 'FN': 194.0, 'LE': 0.0, 'BE': 370.0, 'LBE': 0.0}","{'SINTOMA': {'precision': 0.5276273022751896, 'recall': 0.6536912751677852, 'f1': 0.5839328537170264, 'number': 1490}, 'overall_precision': 0.5276273022751896, 'overall_recall': 0.6536912751677852, 'overall_f1': 0.5839328537170264, 'overall_accuracy': 0.9420021034515728}"
5,No log,0.152791,0.611560,0.709677,"{'SINTOMA': {'precision': 0.7463651050080775, 'recall': 0.6764275256222547, 'f1': 0.7096774193548386, 'trad_prec': 0.6739606126914661, 'trad_rec': 0.6201342281879194, 'trad_f1': 0.6459279972037749, 'TP': 924, 'FP': 173.0, 'FN': 301.0, 'LE': 0.0, 'BE': 282.0, 'LBE': 0.0}, 'overall_precision': 0.7463651050080775, 'overall_recall': 0.6764275256222547, 'overall_f1': 0.7096774193548386, 'overall_trad_prec': 0.6739606126914661, 'overall_trad_rec': 0.6201342281879194, 'overall_trad_f1': 0.6459279972037749, 'TP': 924, 'FP': 173.0, 'FN': 301.0, 'LE': 0.0, 'BE': 282.0, 'LBE': 0.0}","{'SINTOMA': {'precision

eval/loss,█▃▁▁▁▁▂▃▃▃▄▄▄▄▄▅▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
eval/overall_f1,▁▅▆▇▇▇██████████████████████████████████
eval/overall_fair_f1,▁▅▇▇▇███████████████████████████████████
eval/runtime,▁▁▂▁▁▄▂▂▂▁█▅▁▂▂▂▅▂▁▁▂▅▁▁▁▁▂▅▁▁▁▂▄▂▂▂▁▁▆▁
eval/samples_per_second,██▇██▄▇▇▇█▁▄█▇▆▇▄▇██▇▄█▇▇▇▇▄███▇▄▇▇▇██▃█
eval/steps_per_second,██▇██▄▇▇▇▇▁▄█▇▆▇▄▇██▇▄█▇▇▇▇▄███▇▄▇▇▇██▃█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
